In [1]:
# -*- coding: utf-8 -*-
"""
Generated by ArcGIS ModelBuilder on : 2021-09-08 15:13:15 modified by ekelly/CIESIN
"""
import arcpy
import os
import datetime

# Define your inout layers, outputs, geodatabases and other parameters

In [2]:
#DEFINE YOUR PARAMETERS
scratchWorkspace=r"C:\Users\Kelly\Documents\ArcGIS\Projects\DRC_Boundaries_2021\sttlPnts_in_Extents.gdb"
workspace=r"C:\Users\Kelly\Documents\ArcGIS\Projects\DRC_Boundaries_2021\sttlPnts_in_Extents.gdb"
Output_gdb = r"C:\Users\Kelly\Documents\ArcGIS\Projects\DRC_Boundaries_2021\sttlPnts_in_Extents.gdb"

#Input Layers:
Settlement_Extent_Polygons=r"C:\Users\Kelly\Documents\GIS_COLUMBIA\SSD\SSD_Settlements\Extents\GRID3_South_Sudan_Settlement_Extents_Version_1.0\GRID3_South_Sudan_Settlement_Extents_Version_1.0.gdb\GRID3_South_Sudan_Settlement_Extents_Version_1",
Settlement_Points= r"C:\Users\Kelly\Documents\GIS_COLUMBIA\SSD\SSD_Settlements\REACH\ReachMarch2021_Cleaned\ReachSttlPnts_test.shp"

#enter your value for buffer size. The script will search for duplicate names within the settlement extent and its buffer

#the buffer size is layer spacific and depends on the projection you use. 
#Replace in the Expressions below the values for NEAR_DIST that differ from zero with your buffer radius. 
#That means that you have to replace the upper values, not those that already equal zero.Do not change the zero values.
#Leave FREQUENCY > 1 in Expression1. Change only the upper value of the NEAR_DIST

Expression0 = "NEAR_DIST > 0 AND NEAR_DIST < 0.005"
Expression = "NEAR_DIST < 0.005"
Expression1 = "FREQUENCY > 1 And NEAR_DIST > 0 And NEAR_DIST < 0.005"

#enter the field name that contains the settlement names to the right of the equals sign
name_to_check = "name_upd"

#enter thr name of your resulting layer to the right of the equals sign
resulting_layer = "SttlPnts_Cleaned"

#these layers are saved to the scratch geodatabase, but you can fill in any name of your choice on the right side
identical_table = "identical_table"
frequency_table = "frequency_table"

# Run the script

In [3]:
 # To allow overwriting outputs change overwriteOutput option to True.
arcpy.env.overwriteOutput = False
Output = Output_gdb

# Process: Near (Near) (analysis)
arcpy.analysis.Near(in_features=Settlement_Points, near_features=Settlement_Extent_Polygons, search_radius="", location="NO_LOCATION", angle="NO_ANGLE", method="PLANAR", field_names=[["NEAR_FID", "NEAR_FID"], ["NEAR_DIST", "NEAR_DIST"]])[0]
print(arcpy.management.GetCount(Settlement_Points))

    

19553


In [4]:
# Process: Select Layer By Attribute. Selects those points that fall within the settlement extent
# Process: Delete Identical- deletes duplicate names within settlement extents
arcpy.management.DeleteIdentical(in_dataset = arcpy.management.SelectLayerByAttribute(in_layer_or_view=Settlement_Points, selection_type="NEW_SELECTION", where_clause="NEAR_DIST=0", invert_where_clause=""), fields=[name_to_check, "NEAR_FID"], xy_tolerance="", z_tolerance=0)
Updated_Input_Features = arcpy.SelectLayerByAttribute_management(Settlement_Points, "CLEAR_SELECTION")

In [5]:
print(arcpy.management.GetCount(Updated_Input_Features))

19525


In [6]:
# Process: Select Layer By Attribute. Selects those points that fall within the buffer around the settlement extent
# Process: Delete Identical- deletes duplicates within buffers

arcpy.management.DeleteIdentical(in_dataset = arcpy.management.SelectLayerByAttribute(in_layer_or_view=Updated_Input_Features, selection_type="NEW_SELECTION", where_clause= Expression0, invert_where_clause=""), fields=[name_to_check, "NEAR_FID"], xy_tolerance="", z_tolerance=0)
Updated_Input_Features1 = arcpy.SelectLayerByAttribute_management(Updated_Input_Features, "CLEAR_SELECTION")
print(arcpy.management.GetCount(Updated_Input_Features1))



19524


In [7]:
#export layer as feature class
SttlPntsUpd1 = os.path.join(scratchWorkspace, "SttlPntsExport1")
arcpy.management.CopyFeatures(Updated_Input_Features1, SttlPntsUpd1)

#add new reference field to join tables on this additional field later on
arcpy.management.AddField(SttlPntsUpd1, "NEWID", "long", "40000", "", "", "OBJECTID")
arcpy.CalculateField_management(SttlPntsUpd1, "NEWID", '!OBJECTID!')

<Result 'C:\\Users\\Kelly\\Documents\\ArcGIS\\Projects\\DRC_Boundaries_2021\\sttlPnts_in_Extents.gdb\\SttlPntsExport1'>

In [8]:

# Process: Select Layer By Attribute; selects all points within extents and buffers around the extents
selection_new = arcpy.management.SelectLayerByAttribute(in_layer_or_view = SttlPntsUpd1, selection_type="NEW_SELECTION", where_clause = Expression, invert_where_clause="")



In [9]:
# Process: Find Identical (Find Identical) (management). It creates an identical table for the above selected features (within extent and within the 500 meter buffer)

SttlPnts_La_7 = os.path.join(scratchWorkspace, identical_table)
arcpy.management.FindIdentical(in_dataset=selection_new, out_dataset= SttlPnts_La_7, fields=[name_to_check, "NEAR_FID"], xy_tolerance="", z_tolerance=0, output_record_option="ONLY_DUPLICATES")



<Result 'C:\\Users\\Kelly\\Documents\\ArcGIS\\Projects\\DRC_Boundaries_2021\\sttlPnts_in_Extents.gdb\\identical_table'>

In [10]:
# Process: Frequency (Frequency) (analysis). Creates a frequency table based on the above generate identical table.

SttlPnts_La1 = os.path.join(scratchWorkspace, frequency_table)
arcpy.analysis.Frequency(in_table=SttlPnts_La_7, out_table= SttlPnts_La1, frequency_fields=["FEAT_SEQ"])

#arcpy.analysis.Frequency(in_table, out_table, frequency_fields, {summary_fields})

<Result 'C:\\Users\\Kelly\\Documents\\ArcGIS\\Projects\\DRC_Boundaries_2021\\sttlPnts_in_Extents.gdb\\frequency_table'>

In [11]:
arcpy.SelectLayerByAttribute_management(SttlPntsUpd1, "CLEAR_SELECTION")


id,value
0,a Layer object
1,-1


In [12]:
# Process: Join Field (Join Field) (management)
SttlPnts_La_8 = arcpy.management.JoinField(in_data=SttlPnts_La_7, in_field="FEAT_SEQ", join_table=SttlPnts_La1, join_field="FEAT_SEQ")



In [13]:
# Process: Join Field (2) (Join Field) (management)
SttlPnts_La_9 = arcpy.management.JoinField(SttlPntsUpd1, in_field="NEWID", join_table=SttlPnts_La_8, join_field="IN_FID")



In [14]:
# Process: Select Layer By Attribute - (Select Layer By Attribute) (management)
# Process: Delete Rows (Delete Rows) (management)


Updated_Input_With_Rows_Removed = arcpy.management.DeleteRows(in_rows=arcpy.management.SelectLayerByAttribute(in_layer_or_view=SttlPnts_La_9, selection_type="NEW_SELECTION", where_clause=Expression1, invert_where_clause=""))
Updated_Input = arcpy.SelectLayerByAttribute_management(Updated_Input_With_Rows_Removed, "CLEAR_SELECTION")

print(arcpy.management.GetCount(Updated_Input))


19523


In [15]:
#Export your results as resulting layer taht will have the name you defined during Step 1 before running the script
SttlPnts_Cleaned = os.path.join(workspace, resulting_layer)
arcpy.management.CopyFeatures(Updated_Input, SttlPnts_Cleaned)
print(arcpy.management.GetCount(SttlPnts_Cleaned))

19523
